In [46]:
using ReverseDiff
using ProgressMeter
using Base.Test
using MLDataPattern
using Plots
gr()

Plots.GRBackend()

In [93]:
module Nets

import ReverseDiff
using MLDataPattern: batchview, shuffleobs

head(t::Tuple) = tuple(t[1])

function viewblocks{T <: NTuple}(data::AbstractArray, shapes::AbstractVector{T})
    starts = cumsum(vcat([1], prod.(shapes)))
    [reshape(view(data, starts[i]:(starts[i+1] - 1)), shapes[i]) for i in 1:length(shapes)]
end

type PANet{Sensitive}
    shapes::Vector{NTuple{2, Int}}
end

nweights(net::PANet) = sum(prod, net.shapes)
nbiases(net::PANet) = sum(first, net.shapes)
nparams(net::PANet) = nweights(net) + nbiases(net)
Base.rand(net::PANet) = rand(nparams(net))
Base.randn(net::PANet) = randn(nparams(net))

function predict(net::PANet{true}, params::AbstractVector, x::AbstractVector)
    weights = viewblocks(params, net.shapes)
    biases = viewblocks(@view(params[(nweights(net) + 1):end]), head.(net.shapes))
    y = similar(x, Base.promote_eltype(params, x), (length(x), 1))
    y .= x
    J = eye(eltype(y), length(x))
    for i in 1:(length(net.shapes) - 1)
        w = weights[i]
        y = w * y .+ biases[i]
        J = w * J
        z = [yy >= 0 ? 1.0 : 0.1 for yy in y]
        y = y .* z
        J = J .* z
    end
    w = weights[end]
    vcat(vec(w * y), vec(w * J))
end

function predict(net::PANet{false}, params::AbstractVector, x::AbstractVector)
    weights = viewblocks(params, net.shapes)
    biases = viewblocks(@view(params[(nweights(net) + 1):end]), head.(net.shapes))
    y = similar(x, Base.promote_eltype(params, x), (length(x), 1))
    y .= x
    for i in 1:(length(net.shapes) - 1)
        w = weights[i]
        y = w * y .+ biases[i]
        z = [yy >= 0 ? 1.0 : 0.1 for yy in y]
        y = y .* z
    end
    w = weights[end]
    vec(w * y)
end

function sgd!(loss, params, data, lr=0.01, momentum=0.8, batchsize=1)
    last_descent = zeros(params)
    dw = zeros(params)
    dw_sample = zeros(dw)
    for batch in batchview(shuffleobs(data), batchsize)
        dw .= 0
        for (x, y) in batch
            ReverseDiff.gradient!(dw_sample, w -> loss(w, x, y), params)
            if any(isnan.(dw_sample))
                @show params x y dw_sample
                error("nan")
            end
            dw .+= dw_sample
        end
        for i in eachindex(params)
            v = lr * dw[i] + momentum * last_descent[i]
            params[i] -= v
            last_descent[i] = v
        end
    end
    params
end

end

Nets

In [126]:
data = [
begin
    x = randn()
    if x >= 0
        [x], [x, 1]
    else
        [x], [-x, -1]
    end
    end for i in 1:20];
train_data, test_data = splitobs(data, 0.5)


shapes = [(4, 1), (4, 4), (1, 4)]
net = Nets.PANet{true}(shapes)
params = randn(net)
@inferred Nets.predict(net, params, [0])

predict = (params, x) -> Nets.predict(net, params, x)
loss = (params, x, y) -> sumabs2(y - predict(params, x))

@showprogress for i in 1:20
    Nets.sgd!(loss, params, train_data)
end

@show mean(xy -> loss(params, xy[1], xy[2]), test_data)

xx = linspace(-2, 2)
plt = plot(xx, x -> predict(params, [x])[1], ylim=(-1, 1), legend=nothing, linewidth=4)
plot!(plt, [x for (x, y) in train_data], [y for (x, y) in train_data], 
line=nothing,
markershape=:circle,
markercolor=:green,
markersize=3
)
plot!(plt, [x for (x, y) in test_data], [y for (x, y) in test_data], 
line=nothing,
markershape=:circle,
markercolor=:red,
markersize=3
)
plt

Progress:  50%|████████████████████                     |  ETA: 0:00:00

mean((xy->begin  # In[126], line 25:
            loss(params,xy[1],xy[2])
        end),test_data) = 0.07861941889925812


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:3.2; stroke-opacity:1; fill:none" points="
 97.7092,-191.927 107.558,-175.6 117.407,-159.272 127.256,-142.944 137.105,-126.617 146.954,-110.289 156.803,-93.9611 166.652,-77.6333 176.501,-61.3056 186.35,-44.9779 
 196.199,-28.6501 206.048,-12.3224 215.897,4.00532 225.746,20.333 235.595,36.6608 245.444,52.9885 255.293,69.3162 265.142,85.644 274.991,100.265 284.84,112.22 
 294.69,124.175 304.539,136.13 314.388,148.085 324.237,160.04 334.086,162.487 343.935,159.425 353.784,156.363 363.633,149.696 373.482,134.45 383.331,119.204 
 393.18,103.958 403.029,89.067 412.878,74.5195 422.727,59.9721 432.576,45.4246 442.425,30.8771 452.274,16.3296 462.123,1.78218 471.972,-12.7653 481.821,-27.3128 
 491.67,-41.8602 501.519,-56.4077 511.368,-70.9552 521.217,-85.5026 531.066,-100.05 540.915,-114.598 550.764,-129.145 560.613,-143.693 570.462,-158.24 580.311,-172.787 
 
 "/>

In [127]:
data = [
begin
    x = randn()
    if x >= 0
        [x], [x]
    else
        [x], [-x]
    end
    end for i in 1:20];
train_data, test_data = splitobs(data, 0.5)


shapes = [(4, 1), (4, 4), (1, 4)]
net = Nets.PANet{false}(shapes)
params = randn(net)
@inferred Nets.predict(net, params, [0])

predict = (params, x) -> Nets.predict(net, params, x)
loss = (params, x, y) -> sumabs2(y - predict(params, x))

@showprogress for i in 1:10
    Nets.sgd!(loss, params, train_data)
end

@show mean(xy -> loss(params, xy[1], xy[2]), test_data)

xx = linspace(-2, 2)
plt = plot(xx, x -> predict(params, [x])[1], ylim=(-1, 1), legend=nothing, linewidth=4)
plot!(plt, [x for (x, y) in train_data], [y for (x, y) in train_data], 
line=nothing,
markershape=:circle,
markercolor=:green,
markersize=3
)
plot!(plt, [x for (x, y) in test_data], [y for (x, y) in test_data], 
line=nothing,
markershape=:circle,
markercolor=:red,
markersize=3
)
plt

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


mean((xy->begin  # In[127], line 25:
            loss(params,xy[1],xy[2])
        end),test_data) = 0.4778154980705157


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:3.2; stroke-opacity:1; fill:none" points="
 62.7244,15.7007 71.8433,17.6509 80.9621,19.6011 90.081,21.5513 99.1999,23.5015 108.319,25.4517 117.438,27.4019 126.556,29.3521 135.675,31.3022 144.794,33.2524 
 153.913,33.9066 163.032,33.0917 172.151,32.2768 181.27,31.4619 190.389,30.647 199.507,29.8321 208.626,29.0172 217.745,28.2023 226.864,27.3874 235.983,26.5725 
 245.102,25.7576 254.221,24.9427 263.34,24.1278 272.458,23.3129 281.577,22.498 290.696,21.6831 299.815,16.3849 308.934,10.2743 318.053,4.11089 327.172,-2.05256 
 336.291,-8.21601 345.409,-14.3795 354.528,-22.0452 363.647,-30.0746 372.766,-38.104 381.885,-46.1335 391.004,-54.1629 400.123,-62.1924 409.242,-70.2218 418.36,-78.2512 
 427.479,-86.2807 436.598,-94.3101 445.717,-102.34 454.836,-110.369 463.955,-118.398 473.074,-126.428 482.192,-134.457 491.311,-142.487 500.43,-150.516 509.549,-158.546 
 
 "/>